In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('app').getOrCreate()
df = spark.read.csv("mask_data.csv",header=True)
df.show()

+---------------+-------------+------------+-------------------+----------+
|Customer_number|Customer_name|customer_age|              email|    mobile|
+---------------+-------------+------------+-------------------+----------+
|         301053|    Azarudeen|          28|     azar@gmail.com|9842356173|
|         305123|       Sakthi|          29|sakthi.m2@gmail.com|9764510298|
+---------------+-------------+------------+-------------------+----------+



### Email Mask UDF:

In [8]:
def mail_fun(Value):
    mail_usr = Value.split("@")[0]
    n = len(mail_usr)
    ls = list(mail_usr)
    ls[1:int(n)-1] = "*"*int(n-2)
    out = "".join(ls)+"@"+Value.split("@")[1]
    return out

In [9]:
mail_fun("nareshreddy@gmail.com")

'n*********y@gmail.com'

### Mobile Mask UDF 

In [23]:
def mob_fun(val):
    n = len(val)
    ls = list(val)
    ls[2:int(n)-2] = "x"*int(n-4)
    return "".join(ls)

In [24]:
mob_fun("9980528846")

'99xxxxxx46'

### Register UDF

In [25]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

mask_gmail = udf(mail_fun, StringType())
mask_mobile = udf(mob_fun, StringType())

In [26]:
df_mask = df.withColumn("Email_masked",mask_gmail(df["email"])) \
            .withColumn("Mobile_masked",mask_mobile(df["mobile"])) \
            .drop("email","mobile")

In [27]:
df_mask.show()

+---------------+-------------+------------+-------------------+-------------+
|Customer_number|Customer_name|customer_age|       Email_masked|Mobile_masked|
+---------------+-------------+------------+-------------------+-------------+
|         301053|    Azarudeen|          28|     a**r@gmail.com|   98xxxxxx73|
|         305123|       Sakthi|          29|s*******2@gmail.com|   97xxxxxx98|
+---------------+-------------+------------+-------------------+-------------+

